# 023 course2 week3 学习笔记
---

超参优化

1. Try random values:Don't use a grid
2. 

使用对数标尺，而不是线性标尺。

跨领域找灵感


### 如何组织超参优化？
---
babysitting one model
training many models in parallel


### batch normarlization
----
每次只能处理一个mini-batch。


## Softmax回归
----
LR的一般形式。


最大似然估计



这周内容主要分四块：第一部分是超参数，第二部分是batchnorm，第三部分是softmax做多类分类，第四部分是深度学习框架和tensorflow

第一部分首先讲了超参数的优先级，也就是哪些重要、哪些不那么重要
ng认为，超参数的重要性排序如下：
1. learning rate；
2. momentum参数（0.9是不错的默认值和初始值）、mini-batch的batch size、各层的隐藏单元数；
3. 层数和learning rate decay
在使用ADAM优化算法时，ng总是把beta1、beta2、epsilon这三个值直接设置为固定的0.9、0.999、10^(-8)而不去调整它们，也就是说，ng认为调这几个超参数意义不大

ng讲了两种搜索超参数的方法：grid search和random search
grid search在超参数数量较少时比较实用，但random search在超参数数量较多时更实用。原因是，当我们不知道哪些超参数不重要、对结果没什么影响的时候，用grid search能够试验的重要参数上的值要比random search少得多
还有一种搜索超参数的策略是由粗到细（coarse to fine），顾名思义，就是先在大范围内的少数超参数上测试模型性能，然后再在性能比较好的几个超参数的附近区域中采更多超参数进行测试，逐步递进
计算资源多的话就可以多random search一些超参数，后面ng也提到测试更多超参数总是好的

有一些超参数需要做尺度变换，不能直接做均匀随机采样，比如learning rate。因为比如说在0.0001到1之间搜索learning rate，如果均匀随机采样，那么0.1到1之间会占据90%的点，所以这里要用对数尺度下的搜索，先对超参数取值范围的最大最小值取log，然后在log范围内均匀随机采样

指数加权平均里的beta是另一个需要考虑尺度变换的超参数，这个值不能直接取log，因为用的是1-beta而不是beta，所以这里需要对1-beta取log

不过ng也提到，即使没有在合适的尺度上采样，只要采样数足够多，也能获得还算过得去的效果，特别是结合前面讲的由粗到细的搜索策略，在后面较细范围的搜索中还是有机会聚焦到合适的超参数范围上、找到不错的超参数

第二部分是batchnorm，这是非常重要的方法，它让超参数搜索变得更简单，让神经网络选择超参数的过程变得更robust或不那么敏感，工作效果好的超参数范围会变得比之前大了很多，训练非常深的网络也变得容易很多
方法类似于前面讲逻辑回归时做的对每个维度的特征求均值和方差，然后对每个特征做归一化，batchnorm是对mini-batch每一层神经元输出的z做归一化
注意，为了保持数值稳定性，分母根号内需要加上epsilon以防方差为0
但同时，我们并不想让神经元总是均值为0和标准差为1，它们具有不同的分布可能更有意义，所以可以再乘上gamma并加上beta，这两个参数是可学习的，可以在反向传播时更新它们
batchnorm起作用的原因是：可以让网络中较后或较深的层中的权重对前面层的权重变化更鲁棒或不敏感
从后面的层看来，因为前面层的权重在不断更新，所以对于同样的输入，它接收到的输入或前面层给的激活输出都在不断变化，这样就出现了covariate shift问题

batchnorm的作用，就是减小了隐层输出的变化范围，使得不管前面层的权值如何变化，后面层接收到的输入的均值和方差都保持不变
所以即使它们的数值有变化，但由gamma和beta控制的均值和方差却没有变化，限制了前面层参数变化对后面层输入分布的影响，所以后面层就可以在相对固定的分布上进行训练

同时，batchnorm也有轻微的正则化效果
dropout是通过将激活随机设置为0或1来引入噪声，而bn则是因为mini-batch估计均值和方差时带有噪声，这样给前面神经元引入噪声，使得后面神经元训练时不过于依赖于任何一个前面层的神经元
但因为添加的噪声比较微小，所以正则化效果也比较小，实践中可以将bn和dropout一起使用

测试阶段没有mini-batch去估计均值和方差，所以使用指数加权平均在训练阶段的每个batch上计算的均值和方差，作为测试阶段的均值和方差

第三部分是用softmax训练多类分类模型，它实际上是逻辑回归的sigmoid+cross entropy在多类上的推广
前向和反向过程都和逻辑回归比较像，这里就不再赘述了

最后一部分是深度学习框架和tensorflow
ng认为，好的深度学习框架有三个标准：
1. 便于编程，包括神经网络的开发和迭代，以及训练好网络之后对产品做部署
2. 运行速度要快
3. 真正的开放，这指的不仅是开源，而且要有良好的管理，有些公司会逐渐闭源并且把功能转移到他们自己的产品中比如云服务，所以能长时间保持开源、并且不在单一一家公司的掌控之下是很重要的，因为这家公司可能在未来出于某个原因而停止开源

最后ng用一个编程实例来解释了如何用tensorflow定义cost function、如何设置变量和placeholder、如何把输入数据和placeholder绑定起来、如何使用optimizer来做优化，以及构建计算图的概念

covariate shift 怎么理解呢？
那个其实我也不是很理解，说是输入分布变化对训练模型造成的影响


我看了一些batchnorm的解释，包括https://www.zhihu.com/question/38102762 ，说是主要还是解决反向传播时的梯度爆炸和梯度消失问题，说那个什么covariate shift是论文作者瞎解释的，哈哈。。


我漏了一个视频的内容。。补一下，第一部分超参数的最后一小部分，那个熊猫模式和鱼子酱模式

训练模型的过程有两类方法：第一种是在没有足够的计算资源或计算能力时，不能同时试验大量模型，只能照看单一模型，每天都调整这个模型的超参数，让性能变得更好；第二种是同时试验很多不同超参数的模型，最后选择学习曲线看起来效果最好的那个模型
ng形象地把这两种方法称作熊猫模式和鱼子酱模式
ng建议，在计算资源充足的情况下，绝对应该选择鱼子酱模式，充分尝试各种超参数
但他同时也说，在某些领域，比如在线广告和计算机视觉领域，数据很多而且模型很大（层数、节点都很多），很难同时训练很多模型，所以用熊猫模式还是鱼子酱模式也和应用领域相关